# Neural Networks
----
Multi-layer Perceptron (MLP) is a supervised learning algorithm that learns a function $f(\cdot): R^m \rightarrow R^o$ by training on a dataset, where $m$ is the number of dimensions for input and $o$ is the number of dimensions for output. Given a set of features $X = {x_1, x_2, ..., x_m}$ and a target $y$, it can learn a non-linear function approximator for either classification or regression. It is different from logistic regression, in that between the input and the output layer, there can be one or more non-linear layers, called hidden layers. Figure shows a one hidden layer MLP with scalar output.
 <img src="https://sebastianraschka.com/images/faq/visual-backpropagation/backpropagation.png">
 
 ----
 
The advantages of Multi-layer Perceptron are:
- Capability to learn non-linear models.
- Capability to learn models in real-time (on-line learning) using partial_fit.

The disadvantages of Multi-layer Perceptron (MLP) include:
- MLP with hidden layers have a non-convex loss function where there exists more than one local minimum. Therefore different random weight initializations can lead to different validation accuracy.
- MLP requires tuning a number of hyperparameters such as the number of hidden neurons, layers, and iterations.
- MLP is sensitive to feature scaling.

In [1]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Input, BatchNormalization, Dropout, Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model


from sklearn.model_selection import train_test_split
import numpy as np

## Get the data prepared

In [2]:
#Data Split
(train_X, train_Y), (test_X, test_Y) = fashion_mnist.load_data()
train_X, valid_X, train_Y, valid_Y = train_test_split(train_X, train_Y, stratify=train_Y, train_size=7/8)
train_Y = to_categorical(train_Y) # One-hot
valid_Y = to_categorical(valid_Y)
test_Y = to_categorical(test_Y)

4431872/4422102 [==============================] - 0s 0us/step


## MLP Algorithm implement

In [3]:
def MLP(input_shape, neuronsList, opt=Adam(learning_rate=0.001)):
    
    inputs = Input(shape=input_shape)
    
    layer = Flatten()(inputs)
    for i in range(len(neuronsList)):
        layer = Dense(neuronsList[i], activation='relu')(layer)
    
    outputs = Dense(10, activation='softmax')(layer)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Optimizer
adam = Adam(learning_rate=0.001)
#Early stopping to avoid overfitting
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   verbose=1,
                   patience=20,
                   restore_best_weights=True)

# Random search for hyperparameters tuning
nLayers = 3
nIter = 10 # Number of iterations in random search
BestValidLoss = 1. # Best yet validation loss

for i in range(nIter):
    print('\nIteration:', i+1)
    neuronsList = []

    # Randomly choose integers between 16 and 256 for nLayers layers
    for j in range(nLayers):
        neuronsList.append(np.random.randint(low=16, high=257))

    print('# neurons in the layers:', neuronsList)
  
    mlp = MLP(input_shape=train_X[0].shape, neuronsList=neuronsList, opt=adam)
    history = mlp.fit(train_X, train_Y, epochs=80, batch_size=64, validation_data=(valid_X, valid_Y), verbose=0, callbacks=[es])

    # validation
    validLoss, validAcc = mlp.evaluate(valid_X, valid_Y, verbose=0)

    print('Validation loss:{:0.5f}'.format(validLoss))

    # Store best model and history
    if validLoss < BestValidLoss:
        print('Updating best yet model...')
        BestValidLoss = validLoss
        BestModel = mlp
        History = history


Iteration: 1
# neurons in the layers: [100, 178, 147]
Restoring model weights from the end of the best epoch: 12.
Epoch 00032: early stopping
Validation loss:0.37043
Updating best yet model...

Iteration: 2
# neurons in the layers: [252, 143, 118]
Restoring model weights from the end of the best epoch: 14.
Epoch 00034: early stopping
Validation loss:0.36231
Updating best yet model...

Iteration: 3
# neurons in the layers: [227, 193, 21]
Restoring model weights from the end of the best epoch: 6.
Epoch 00026: early stopping
Validation loss:2.30259

Iteration: 4
# neurons in the layers: [138, 18, 175]
Restoring model weights from the end of the best epoch: 14.
Epoch 00034: early stopping
Validation loss:2.30260

Iteration: 5
# neurons in the layers: [105, 63, 172]
Restoring model weights from the end of the best epoch: 25.
Epoch 00045: early stopping
Validation loss:0.38912

Iteration: 6
# neurons in the layers: [164, 137, 161]
Restoring model weights from the end of the best epoch: 15.


In [4]:
# evaluate model on test set
test_a = BestModel.evaluate(test_X,test_Y,verbose=1)

313/313 [==============================] - 1s 4ms/step - loss: 0.3680 - accuracy: 0.8751


## Conclusion
After ten iterations, the accuracy of the model reached 87.5%